# ASU / IllumiDesk LTI 1.3 Testing Notes

## Overview

This document summarizes the test results when testing the **LTI 1.3 authentication flow between Arizona State University's Canvas LMS instance and the IllumiDesk external tool**. Additionally, this document provides some action items to streamline the installation process and to better handle edge cases.

## IllumiDesk's Current Authentication Workflow with Canvas / LTI 1.3

The first step is to [install the IllumiDesk external tool](https://docs.illumidesk.com/v2.0) within the Canvas LMS and activate the tool within a course. Once these tasks are completed, users are able to launch the external tool from the course navigation menu or by clicking on assignment link(s).

The authentication workflow consists to three general steps:

1. Decode payload with private/public key pairs
2. Validate the request payload as LTI 1.3 compliant
3. Obtain email address from user's payload and associate it to the username
4. Import users enrolled in the course

Regarding the `username` task, the IllumiDesk application uses the following rules to associate an identifier to the `username`:

1. Decode request payload and dump to dictionary (JSON)
2. Associate the username by normalizing the characters within the user's email address before the `@` symbol. For example, if the user's email is `foo_bar.12@example.com` the username would be `foobar12`.

This snippet provides an example of how the email value is obtained from the `email` key which is then passed to the `normalize_email` function:

```python
username = normalize_email(decoded_json_payload['email'])
```

If the email is not present of there is an error normalizing the email then the application will return a `500 error` to the user.


## Testing Summary

During testing, additional logs were added to capture the raw LTI 1.3 compatible request payload after it was successfully decoded. The original result is attached in the `lti13_request.json` document.

With a valid `json` document stored in memory the application is then able to search of `keys` within the json document and obtain the `values` associated to those `keys`.

### Key/Value Pairs in Decoded Payload

Many methods are available to view specific `key/value` pairs within the JSON object. The examples below are provided in **Python 3**.

Let us print the original payload to get a feel for the data provided in the `JSON` format.

In [1]:
import json

with open('lti13_request.json') as f:
    
    # Load the json data form a file
    data = json.load(f)

    # Pretty Printing JSON string back
    print(json.dumps(data, indent = 4, sort_keys=True))

{
    "aud": "127470000000000057",
    "azp": "127470000000000057",
    "email": "suave77@asu.edu",
    "errors": {
        "errors": {}
    },
    "exp": 1583869352,
    "family_name": "EX",
    "given_name": "Luke William Dorsett",
    "https://purl.imsglobal.org/spec/lti-ags/claim/endpoint": {
        "errors": {
            "errors": {}
        },
        "lineitems": "https://asu-dev.instructure.com/api/lti/courses/249/line_items",
        "scope": [
            "https://purl.imsglobal.org/spec/lti-ags/scope/lineitem",
            "https://purl.imsglobal.org/spec/lti-ags/scope/lineitem.readonly",
            "https://purl.imsglobal.org/spec/lti-ags/scope/result.readonly",
            "https://purl.imsglobal.org/spec/lti-ags/scope/score"
        ],
        "validation_context": ""
    },
    "https://purl.imsglobal.org/spec/lti-nrps/claim/namesroleservice": {
        "context_memberships_url": "https://asu-dev.instructure.com/api/lti/courses/249/names_and_roles",
        "errors": 

```json
{
    "aud": "127470000000000057",
    "azp": "127470000000000057",
    "email": "suave77@asu.edu",
    "errors": {
        "errors": {}
    },
    "exp": 1583869352,
    "family_name": "EX",
    "given_name": "Luke William Dorsett",
    "https://purl.imsglobal.org/spec/lti-ags/claim/endpoint": {
        "errors": {
            "errors": {}
        },
        "lineitems": "https://asu-dev.instructure.com/api/lti/courses/249/line_items",
        "scope": [
            "https://purl.imsglobal.org/spec/lti-ags/scope/lineitem",
            "https://purl.imsglobal.org/spec/lti-ags/scope/lineitem.readonly",
            "https://purl.imsglobal.org/spec/lti-ags/scope/result.readonly",
            "https://purl.imsglobal.org/spec/lti-ags/scope/score"
        ],
        "validation_context": ""
    },
    "https://purl.imsglobal.org/spec/lti-nrps/claim/namesroleservice": {
        "context_memberships_url": "https://asu-dev.instructure.com/api/lti/courses/249/names_and_roles",
        "errors": {
            "errors": {}
        },
        "service_versions": [
            "2.0"
        ],
        "validation_context": ""
    },
    "https://purl.imsglobal.org/spec/lti/claim/context": {
        "errors": {
            "errors": {}
        },
        "id": "c55d6401fdf909a225cc53a8d7c4994e5fc77506",
        "label": "DEV-Illumidesk2020",
        "title": "Illumidesk LTI 1.3",
        "type": [
            "http://purl.imsglobal.org/vocab/lis/v2/course#CourseOffering"
        ],
        "validation_context": ""
    },
    "https://purl.imsglobal.org/spec/lti/claim/custom": {},
    "https://purl.imsglobal.org/spec/lti/claim/deployment_id": "539:c55d6401fdf909a225cc53a8d7c4994e5fc77506",
    "https://purl.imsglobal.org/spec/lti/claim/launch_presentation": {
        "document_target": "iframe",
        "errors": {
            "errors": {}
        },
        "height": 400,
        "locale": "en",
        "return_url": "https://asu-dev.instructure.com/courses/249/external_content/success/external_tool_redirect",
        "validation_context": "",
        "width": 800
    },
    "https://purl.imsglobal.org/spec/lti/claim/lis": {
        "course_offering_sourcedid": "",
        "errors": {
            "errors": {}
        },
        "person_sourcedid": "ex_suave77",
        "validation_context": ""
    },
    "https://purl.imsglobal.org/spec/lti/claim/message_type": "LtiResourceLinkRequest",
    "https://purl.imsglobal.org/spec/lti/claim/resource_link": {
        "description": "",
        "errors": {
            "errors": {}
        },
        "id": "c55d6401fdf909a225cc53a8d7c4994e5fc77506",
        "title": "",
        "validation_context": ""
    },
    "https://purl.imsglobal.org/spec/lti/claim/roles": [
        "http://purl.imsglobal.org/vocab/lis/v2/institution/person#Administrator",
        "http://purl.imsglobal.org/vocab/lis/v2/institution/person#Instructor",
        "http://purl.imsglobal.org/vocab/lis/v2/membership#Instructor",
        "http://purl.imsglobal.org/vocab/lis/v2/system/person#User"
    ],
    "https://purl.imsglobal.org/spec/lti/claim/target_link_uri": "https://asu.illumnidesk.com/hub/home",
    "https://purl.imsglobal.org/spec/lti/claim/tool_platform": {
        "errors": {
            "errors": {}
        },
        "guid": "kucFCdgLqdHHdMFwSbgH61ebjrRqrAW9ycKVg2OY:canvas-lms",
        "name": "Arizona State University - Dev",
        "product_family_code": "canvas",
        "validation_context": "",
        "version": "cloud"
    },
    "https://purl.imsglobal.org/spec/lti/claim/version": "1.3.0",
    "iat": 1583865752,
    "iss": "https://canvas.instructure.com",
    "locale": "en",
    "name": "Luke William Dorsett EX",
    "nonce": "123837514805708872371583865753",
    "picture": "https://canvas.instructure.com/images/messages/avatar-50.png",
    "sub": "6380f84d-26be-4d87-8219-bb08650efd07"
}
```

There are several important keys available, but for the purposes of these tests we are interested in:
    
1. Keys/Values that uniquely identify the user within the Canvas LMS course
2. User's role (Instructor or Student)

Therefore to obtain a user's email address, we could do something like so:

In [2]:
with open('lti13_request.json') as f:
    
    # Load the json data form a file
    data = json.load(f)
    
    # Get the user's email
    username = data['email']
    
    # Print the username
    print(username)

suave77@asu.edu


It is also simple to obtain LTI 1.3 compatible items. For example, to obtain the user's role you can fetch a nested key/values from the decoded payload to verify the authenticated user's role:

In [3]:
with open('lti13_request.json') as f:
    
    # Load the json data form a file
    data = json.load(f)
    
    # Associate the user to the Learner role by default
    user_role = 'Learner'
    
    # Check user's role
    if ('http://purl.imsglobal.org/vocab/lis/v2/membership#Instructor' 
        in data['https://purl.imsglobal.org/spec/lti/claim/roles']):
        user_role = 'Instructor'
    
    # Print the username
    print(user_role)

Instructor


### Errors Encoutered

Errors were encountered primarily when fetching key/values from the decoded payload to set the `username`. Also, there were some request errors when fetching all user's from the Canvas LMS course. These errors seem to reflect edge cases IllumiDesk has not encountered with their own Canvas LMS development environment hosted at https://illumidesk.instructure.com.


## Action Items

### IllumiDesk

There are two general items:

1. Activate the IllumIDesk application with LTI 1.1
2. Update the LTI 1.3 authentication and user import flow

The LTI 1.1 development environment has been provisioned, therefore item 2 above is in the pending state.

IllumiDesk is in the process of adjusting the authentication flow to set the username to use the `person_sourcedid` by the evening of March 15th. Additionally, the tasks associated with importing user data from the Canvas LMS course are being refactored to better separate concerns so that errors during the user fetch tasks do not affect the authentication flow.

> **Note**: users and their respective roles are imported to enable Auto-Grading features and so Instructors and post grades from the external tool to the LMS.

Below is an example of how to fetch the `person_sourcedid` value:

In [4]:
with open('lti13_request.json') as f:
    
    # Load the json data form a file
    data = json.load(f)
    
    # Associate the user to the Learner role by default
    user_role = 'Learner'
    
    # Check user's role
    if ('http://purl.imsglobal.org/vocab/lis/v2/membership#Instructor' 
        in data['https://purl.imsglobal.org/spec/lti/claim/roles']):
        user_role = 'Instructor'
    
    # Print the username
    print(user_role)

Instructor


### Arizon State University

Validate implementation with LTI 1.1 and notify IllumiDesk when further testing with LTI 1.3 is available.